In [ ]:
import cv2
import mediapipe as mp

# Initialize drawing and face mesh tools
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Load the image
image = cv2.imread('path/to/your/image.jpg')

# Convert the image to RGB format (OpenCV uses BGR)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Create the face mesh object
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    # Analyze the image
    results = face_mesh.process(rgb_image)

    # Extract landmarks if faces are found
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Loop through each landmark
            for landmark in face_landmarks.landmark:
                # Get landmark x and y coordinates
                landmark_x = landmark.x
                landmark_y = landmark.y

                # You can store or process these coordinates here
                print(f"Landmark ({landmark.landmark_type}) coordinates: ({landmark_x}, {landmark_y})")




In [ ]:
import os
import dlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from PIL import Image

# Path to directory containing images
image_directory = 'path/to/your/image/directory'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def plot_delaunay(points, delaunay, ax):
    for simplex in delaunay.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], 'k-')

def detect_faces_and_triangulate(image_path):
    img = Image.open(image_path)
    img = np.array(img)

    ax = plt.gca()
    ax.imshow(img)

    detections = detector(img, 1)
    for k, d in enumerate(detections):
        shape = predictor(img, d)
        points = np.array([[p.x, p.y] for p in shape.parts()])
        ax.scatter(points[:, 0], points[:, 1], c='r', s=10)
        delaunay = Delaunay(points)
        plot_delaunay(points, delaunay, ax)    

    plt.axis('off')
    plt.show()

for image_file in os.listdir(image_directory):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_directory, image_file)
        detect_faces_and_triangulate(image_path)


In [ ]:
import os
import dlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from PIL import Image

# Path to directory containing images
image_directory = 'path/to/your/image/directory'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def plot_delaunay(points, delaunay, ax):
    for simplex in delaunay.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], 'k-')


def get_delaunay_for_part(points, part_indices):
    """
    Get the Delaunay triangulation for a specific part of the face.

    Args:
        points (np.array): 68 facial landmarks points.
        part_indices (list): Indices of the points that belong to the specific part of the face.

    Returns:
        Delaunay: Delaunay triangulation for the specific part of the face.
    """
    part_points = points[part_indices]
    return Delaunay(part_points)  


def detect_faces_and_triangulate_parts(image_path):
    img = Image.open(image_path)
    img = np.array(img)

    ax = plt.gca()
    ax.imshow(img)

    detections = detector(img, 1)
    for k, d in enumerate(detections):
        shape = predictor(img, d)
        points = np.array([[p.x, p.y] for p in shape.parts()])
        ax.scatter(points[:, 0], points[:, 1], c='r', s=10)

        # Define the indices of facial landmarks for each part of the face
        eye_indices = np.arange(36, 48)
        lip_indices = np.arange(48, 68)

        # Get the Delaunay triangulation for each part of the face
        eye_delaunay = get_delaunay_for_part(points, eye_indices)
        lip_delaunay = get_delaunay_for_part(points, lip_indices)

        # Plot Delaunay triangulations
        plot_delaunay(points, eye_delaunay, ax)
        plot_delaunay(points, lip_delaunay, ax)

    plt.axis('off')
    plt.show()

for image_file in os.listdir(image_directory):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_directory, image_file)
        detect_faces_and_triangulate_parts(image_path)


In [ ]:
import cv2

# Load the pre-trained face cascade classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface.xml')

def detect_faces(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Return the number of faces detected
    return len(faces)

# Example usage
image_path = 'path/to/your/image.jpg'
num_faces = detect_faces(image_path)

if num_faces > 0:
    print(f"Image contains {num_faces} faces")
else:
    print("No faces detected in image")


In [ ]:
def filter_image(image_path, has_face):
    num_faces = detect_faces(image_path)
    return num_faces == has_face  # True if face count matches the requirement

# Example usage: Filter for images with at least one face
filtered_images = [image for image in image_paths if filter_image(image, True)]

print("Paths to images with at least one face:")
for path in filtered_images:
    print(path)
